In [1]:
# notebook for processing gutenberg, wikipedia, fineweb large scale chunk data
from datasets import Dataset, load_dataset
from setretrieval.datagen.data_utils import filter_guten_docdata

/accounts/projects/sewonm/prasann/.conda/envs/scaling2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def process_gutenberg_chunks(intext, idval=None):
    divided = intext.split("\n\n")
    results = []
    for d in divided:
        if len(d) < 100:
            continue
        if len(d) > 2000:
            # split by sentences
            restmp = d.replace("\n", " ").split('.')
            # join every 5 sentences
            restmp = [restmp[i:i+10] for i in range(0, len(restmp), 5)]
            results.extend([". ".join(r).strip()[:2000] for r in restmp])
        else:
            results.append(d.replace("\n", " ").strip())
    return [{'text': r, 'wc': len(r.split()), 'id': idval} for r in results]


def group_chunks_maxsize(clist, msize=200):
    # group chunks into group of at least msize
    newclist = []
    cur = {'text': '', 'id': '', 'wc': 0}
    for c in clist: 
        if cur['wc'] + c['wc'] <= msize: 
            cur['text'] += c['text']
            cur['wc'] += c['wc']
            cur['id'] = c['id']
        else: 
            newclist.append(cur)
            cur = c
    newclist.append(cur)
    return newclist

In [31]:
# load in wikipedia chunks
wikichunkshuge = Dataset.load_from_disk("../propercache/data/datastores/wikipedia_docs_1.5M/").shuffle(seed=67)

In [32]:
def filter_wikichunks(chunk):
    if len(chunk['text'].split()) < 80 or chunk['text'].count(".") < 4:
        return False
    if chunk['text'].count("\{") > 3 or chunk['text'].count("\n") > 5:
        return False
    return True

wikichunkshuge = wikichunkshuge.filter(filter_wikichunks, num_proc=10)
print(len(wikichunkshuge))

Filter (num_proc=10):   0%|          | 0/1575261 [00:00<?, ? examples/s]

Filter (num_proc=10): 100%|██████████| 1575261/1575261 [00:02<00:00, 670236.95 examples/s]


209492


In [34]:
wikichunkshuge.save_to_disk("../propercache/data/datastores/wikipedia_docs_200k_cleaned")

Saving the dataset (1/1 shards): 100%|██████████| 209492/209492 [00:02<00:00, 70542.96 examples/s] 


In [33]:
for i in range(100):
    print(wikichunkshuge[i]['text'])
    print("------")

In the field of epidemiology, the causal mechanisms responsible for diseases can be understood using the causal pie model.This conceptual model was introduced by Ken Rothman to communicate how constellations of component causes can lead to a sufficient cause to lead to a condition of interest and that reflection on these sets could improve epidemiological study design. A set of proposed causal mechanisms are represented as pie charts where each pie in the diagram represent a theoretical causal mechanism for a given disease, which is also called a sufficient cause.  Each pie is made up of many component factors, otherwise known as component causes represented by sectors in the diagram. In this framework, each component cause represents an event or condition required for a given disease or outcome. A component cause that appears in every pie is called a necessary cause as the outcome cannot occur without it.
------
Without warning, Armenian soldiers attack the fortress, as snow is fallin

In [3]:
gutenbergdata = Dataset.load_from_disk("../propercache/data/datastores/gutenberg_1k")

In [4]:
fullgutenbergen = load_dataset("manu/project_gutenberg", split="en")

In [5]:
ignoreids = set(gutenbergdata['id'])

In [7]:
fullgutenbergen = fullgutenbergen.filter(lambda row: row['id'] not in ignoreids, num_proc=1)

Filter (num_proc=1):  23%|██▎       | 14000/61340 [00:35<02:01, 388.92 examples/s]


TimeoutError: 